<a href="https://colab.research.google.com/github/jindaldisha/Deep-Learning-and-Neural-Networks/blob/main/Neural-Networks-with-Tensorflow/00_03_gradient_descent_and_linear_regression_with_pytorch_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Gradient Descent and Linear Regression with PyTorch from scratch

##Introduction to Linear Regression
Linear Regression is one of the foundational algorithms in Machine Learning.

We'll create a model that predicts crop yields for apples and oranges (target variables) by looking at the average temperature, rainfall, and humidity (input variables or features) in a region.

Data:

| Region | Temp (F) | Rainfall (mm) | Humidity (%) | Apples (ton) | Oranges (ton) |
| -- | -- | -- | -- | -- | -- |
| Kanto | 73 | 67 | 43 | 56 | 70 |
| Johto | 91 | 88 | 64 | 81 | 101 | 
| Hoenn | 87 | 134 | 58 | 119 | 133 | 
| Sinnoh | 102 | 43 | 37 | 22 | 37 | 
| Unova | 69 | 96 | 70 | 103 | 119 |  

In a linear regression model, each target variable is estimated to be a weighted (w) sum of the input variables, offset by some constant (b), known as a bias :

`yield_apple = w11 * temp + w12 * rainfall + w13 * humidity + b1`

`yield_oranges = w21 * temp + w22 * rainfall + w23 * humidity + b2`

The learning part of linear regression is to figure out the weights (w11, w12..) and the biases (b1, b2..) using the training data and make accurate predictions for the test data that we pass to it. The learned weights will be used to predict the yields for apples and oranges in new regions using the average temperature, rainfall and humidity for that region.

We train our model by adjusting the weights and biases slighty many time and keep improving our accuracy. To improve our model, we use an optimization technique called **Gradient Descent**.


In [1]:
#Import Libraries
import numpy as np
import torch

##Training data

We'll represent the training data using matrics x_train and y_train.

x_train will have the values of temperature, rainfall and humidity for every region as a single row each.
y_train will have the crop yields of apples and oranges for every region as a single row each.

In [2]:
#Input Features (Temp, Rainfall, Humidity)
x_train = np.array([[73, 67, 43], #Kanto
                    [91, 88, 64], #Johto
                    [87, 134, 58], #Hoenn
                    [102, 43, 37], #Sinnoh
                    [69, 96, 70]], #Unova
                    dtype= 'float32')
x_train

array([[ 73.,  67.,  43.],
       [ 91.,  88.,  64.],
       [ 87., 134.,  58.],
       [102.,  43.,  37.],
       [ 69.,  96.,  70.]], dtype=float32)

In [3]:
# Input Labels (Apples, Oranges)
y_train = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')
y_train

array([[ 56.,  70.],
       [ 81., 101.],
       [119., 133.],
       [ 22.,  37.],
       [103., 119.]], dtype=float32)

In [4]:
#Convert input numpy array to pytorch tensors
x_train = torch.from_numpy(x_train)
y_train = torch.from_numpy(y_train)

In [5]:
x_train, type(x_train), x_train.shape

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.],
         [102.,  43.,  37.],
         [ 69.,  96.,  70.]]), torch.Tensor, torch.Size([5, 3]))

In [6]:
y_train, type(y_train), y_train.shape

(tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.],
         [ 22.,  37.],
         [103., 119.]]), torch.Tensor, torch.Size([5, 2]))

## Linear regression model from scratch

The weights and biases (`w11, w12,... w23, b1 & b2`) can also be represented as matrices, initialized as random values. The first row of `w` and the first element of `b` are used to predict the first target variable, i.e., yield of apples, and similarly, the second for oranges.

In [7]:
w = torch.randn(2,3, requires_grad = True)
b = torch.randn(2, requires_grad = True)
w, b

(tensor([[ 1.2442, -1.2817, -1.7407],
         [ 0.7963,  2.1885, -1.1844]], requires_grad=True),
 tensor([ 0.0167, -0.0986], requires_grad=True))

Our Model is going to perform matrix multiplication of input features `x_train` and weights `w` (transposed) and add biases `b`.

torch.randn creates a tensor with the given shape, with elements picked randomly from a normal distribution with mean 0 and standard deviation 1.

In [8]:
#Function for model
def model(x):
  return x @ w.t() + b

In [9]:
#Calling the model on our input featurs
y_pred = model(x_train)

In [10]:
#Comparing y_pred with y_train
y_pred, y_train

(tensor([[ -69.8795,  153.7324],
         [-110.9540,  189.1523],
         [-164.4466,  293.7462],
         [   7.4091,  131.4058],
         [-159.0251,  182.0362]], grad_fn=<AddBackward0>),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.],
         [ 22.,  37.],
         [103., 119.]]))

There is a big difference between our model's predicted labels and the true labels. This is because we've initializes our model with random weights and biases.

##Loss Function

Before we make changes in our model to improve our predictions, we need a way to evaluate our model. We can compare the predicted labels with true labels using the following steps"
- Calculate the difference between `y_pred` and `y_train`
- Take a square of all the differences
- Calculate the average of the elements.

The result is a single numbers and this is know as the `mean square error (MSE)`. It is mean of the square of the difference in the predicted value and the true value.

On average, each element in the prediction differs from the actual target by the square root of the loss. The result is called the loss because it indicates how bad the model is at predicting the target variables. It represents information loss in the model: the lower the loss, the better the model.

In [11]:
#Loss Function (MSE)
def mse(y_true, y_pred):
  error = y_true - y_pred
  square = error * error
  number_of_elements = error.numel()
  mean = torch.sum(square) / number_of_elements
  return mean

In [12]:
#Compute Loss
loss = mse(y_train, y_pred)
print(loss)

tensor(25541.1328, grad_fn=<DivBackward0>)


## Compute gradients

With PyTorch, we can automatically compute the gradient or derivative of the loss w.r.t. to the weights and biases because they have `requires_grad` set to `True`.

The gradients are stored in the `.grad` property of the respective tensors. Note that the derivative of the loss w.r.t. the weights matrix is itself a matrix with the same dimensions.

In [13]:
#Compute Gradients
loss.backward()

In [14]:
#Gradients for weights
print(w)
print(w.grad) 

tensor([[ 1.2442, -1.2817, -1.7407],
        [ 0.7963,  2.1885, -1.1844]], requires_grad=True)
tensor([[-14176.9756, -17817.9082, -10603.8789],
        [  8419.6279,   9003.6797,   5294.2148]])


Since we've calculated gradients for the loss.
`w.grad` is the derivative of the `loss` w.r.t to `w`.

In [15]:
print(b)
print(b.grad)

tensor([ 0.0167, -0.0986], requires_grad=True)
tensor([-175.5792,   98.0146])


## Adjust Weights and Biases to reduce the Loss

The loss is function of our weights and biases and we want to find the set of weights where the loss is the lowest. 

The gradients indicate the rate of change of the loss, i.e. the loss function's slope w.r.t the weights and biases.

If the gradient element is positive:
 - Increasing the weight element's value slightly will increase the loss.
 - Decreasing the weight element's value will decrease the loss.

If the gradient element is negative:
 - Increasing the weight element's value slightly will decrease the loss.
 - Decreasing the weight element's value will increase the loss.


The increase or decrease in the loss by changing a weight element is proportional to the gradient of the loss w.r.t. that element. This observation forms the basis of the gradient descent optimization algorithm that we'll use to improve our model (by descending along the gradient).

We can subtract from each weight element a small quantity proportional to the derivative of the loss w.r.t. that element to reduce the loss slightly.


In simple words, if the gradient is negative, the slope is negative so we need to increase the weight. If the gradient is positive, the slope is positive so we need to decrease the weight.

It is called gradient descent, because we're descending along the gradient.

In [16]:
with torch.no_grad():
  w -= w.grad * 1e-5
  b -= b.grad * 1e-5

We multiply the gradients with a very small number to ensure that we don't modify the weights by a very large amount. This ensure that we only take small steps in the downhill direction and not giant leaps. This prevents from diverging away from the optimal solution. This number is known as the learning rate. 
We use torch.no_grad to we shouldn't track, calculate or modify the gradients while updating the weights and biases.

In [17]:
#Evaluate the model again
y_pred = model(x_train)
loss = mse(y_train, y_pred)
print(loss)

tensor(18155.1465, grad_fn=<DivBackward0>)


Before we proceed, we reset the gradients to zero by invoking the `.zero_()` method. We need to do this because PyTorch accumulates gradients. Otherwise, the next time we invoke `.backward` on the loss, the new gradient values are added to the existing gradients, which may lead to unexpected results.

In [18]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


## Train the model using gradient descent

We reduce the loss and improve our model using the gradient descent optimization algorithm. Thus, we can train the model using the following steps:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero


In [19]:
# Step 1: Generate Predictions
y_pred = model(x_train)
y_pred

tensor([[ -43.0309,  139.2761],
        [ -75.5849,  170.1779],
        [-122.0847,  271.2846],
        [  33.4565,  116.9863],
        [-124.7134,  163.8763]], grad_fn=<AddBackward0>)

In [20]:
# Step 2: Calculate Loss
loss = mse(y_pred, y_train)
loss

tensor(18155.1465, grad_fn=<DivBackward0>)

In [21]:
#Step 3:Compute Gradients
loss.backward()

In [22]:
print(w.grad)
print(b.grad)

tensor([[-11399.3018, -14817.5488,  -8755.7441],
        [  6927.6348,   7401.3652,   4305.5195]])
tensor([-142.5915,   80.3202])


In [23]:
#Step 4, 5: Adjust Weights and Reset Gradients
with torch.no_grad():
  w -= w.grad * 1e-5
  b -= b.grad * 1e-5
  w.grad.zero_()
  b.grad.zero_()

In [24]:
#View new weights and biases
w, b

(tensor([[ 1.5000, -0.9554, -1.5471],
         [ 0.6428,  2.0245, -1.2804]], requires_grad=True),
 tensor([ 0.0199, -0.1003], requires_grad=True))

In [25]:
#Evaluate the Model again
y_pred = model(x_train)
loss = mse(y_train, y_pred)
print(loss)

tensor(13167.5908, grad_fn=<DivBackward0>)


##Train for multiple epochs

To reduce the loss further, we repeat the process of adjusting the weights and biases using the gradient descent algorithm multiple times. Each iteration is called an epoch.

In [26]:
#Train for 100 epochs
for i in range(100):
  #Make Prediction
  y_pred = model(x_train)
  #Calculate Loss
  loss = mse(y_train, y_pred)
  #Print Loss at each epoch
  print(f'Loss at Epoch {i}: {loss}')
  #Calculate Gradients
  loss.backward()
  #Gradient Descent
  with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    #Reset gradients to zero
    w.grad.zero_()
    b.grad.zero_()


Loss at Epoch 0: 13167.5908203125
Loss at Epoch 1: 9796.4248046875
Loss at Epoch 2: 7514.6533203125
Loss at Epoch 3: 5967.12548828125
Loss at Epoch 4: 4914.52001953125
Loss at Epoch 5: 4195.5517578125
Loss at Epoch 6: 3701.538330078125
Loss at Epoch 7: 3359.23388671875
Loss at Epoch 8: 3119.281005859375
Loss at Epoch 9: 2948.415283203125
Loss at Epoch 10: 2824.218017578125
Loss at Epoch 11: 2731.581787109375
Loss at Epoch 12: 2660.323974609375
Loss at Epoch 13: 2603.5810546875
Loss at Epoch 14: 2556.72705078125
Loss at Epoch 15: 2516.642578125
Loss at Epoch 16: 2481.224609375
Loss at Epoch 17: 2449.053466796875
Loss at Epoch 18: 2419.174560546875
Loss at Epoch 19: 2390.937744140625
Loss at Epoch 20: 2363.9091796875
Loss at Epoch 21: 2337.79150390625
Loss at Epoch 22: 2312.38623046875
Loss at Epoch 23: 2287.5546875
Loss at Epoch 24: 2263.20556640625
Loss at Epoch 25: 2239.2734375
Loss at Epoch 26: 2215.715576171875
Loss at Epoch 27: 2192.5
Loss at Epoch 28: 2169.60498046875
Loss at Epoc

As it can be seen from above, the loss decreases with each epoch.

In [27]:
y_pred

tensor([[ 70.1820,  72.5652],
        [ 80.0558,  84.5284],
        [102.6588, 166.0425],
        [ 95.6146,  48.4494],
        [ 54.6875,  84.3634]], grad_fn=<AddBackward0>)

In [28]:
y_train

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

The predicted values are closer to the true values as compared to before.